# Sports Savvy #

The goal of this product is to predict the performance of players in a given game. To accomplish this goal, our software will use past news and other available information and consider how similar conditions affected a player's past game. From there, it will calculate the probability of the player making his daily fantasy line.

In [46]:
import numpy as np
import pandas as pd

In [47]:
cd "C:\Users\IFE\OneDrive - eastern.edu\Documents\Fall 2023\CSCI-330B\NBA Players Stats"

C:\Users\IFE\OneDrive - eastern.edu\Documents\Fall 2023\CSCI-330B\NBA Players Stats


Importing dataset to analyze

In [48]:
player_stats_2019_2020 = pd.read_csv('2019-20_pbp.csv')
player_stats_2019_2020.head()

,URL,GameType,Location,Date,Time,WinningTeam,Quarter,SecLeft,AwayTeam,AwayPlay,...,EnterGame,LeaveGame,TurnoverPlayer,TurnoverType,TurnoverCause,TurnoverCauser,JumpballAwayPlayer,JumpballHomePlayer,JumpballPoss,Unnamed: 40
0,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,720,NOP,Jump ball: D. Favors vs. M. Gasol (L. Ball gai...,...,NaN,NaN,NaN,NaN,NaN,NaN,D. Favors - favorde01,M. Gasol - gasolma01,L. Ball - balllo01,NaN
1,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,708,NOP,L. Ball misses 2-pt jump shot from 11 ft,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,707,NOP,Offensive rebound by D. Favors,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,707,NOP,D. Favors makes 2-pt layup at rim,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,689,NOP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Dropping unnecessary columns and filling missing values with 0

In [49]:
# Columns to keep
keep_columns = ['Location', 'Date', 'WinningTeam', 'AwayTeam', 'HomeTeam', 'Shooter', 'ShotType', 'ShotOutcome', 'Assister', 'Rebounder', 'FreeThrowShooter', 'FreeThrowOutcome', 'TurnoverPlayer']

# Identify columns to drop
drop_columns = set(player_stats_2019_2020.columns) - set(keep_columns)

# Drop columns not in the specified set
player_stats_2019_2020.drop(columns = drop_columns, inplace = True)

player_stats_2019_2020.fillna(0, inplace = True)
player_stats_2019_2020

,Location,Date,WinningTeam,AwayTeam,HomeTeam,Shooter,ShotType,ShotOutcome,Assister,Rebounder,FreeThrowShooter,FreeThrowOutcome,TurnoverPlayer
0,Scotiabank Arena Toronto Canada,October 22 2019,TOR,NOP,TOR,0,0,0,0,0,0,0,0
1,Scotiabank Arena Toronto Canada,October 22 2019,TOR,NOP,TOR,L. Ball - balllo01,2-pt jump shot,miss,0,0,0,0,0
2,Scotiabank Arena Toronto Canada,October 22 2019,TOR,NOP,TOR,0,0,0,0,D. Favors - favorde01,0,0,0
3,Scotiabank Arena Toronto Canada,October 22 2019,TOR,NOP,TOR,D. Favors - favorde01,2-pt layup,make,0,0,0,0,0
4,Scotiabank Arena Toronto Canada,October 22 2019,TOR,NOP,TOR,O. Anunoby - anunoog01,2-pt layup,miss,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
539260,The Arena Bay Lake Florida,October 11 2020,LAL,LAL,MIA,K. Olynyk - olynyke01,2-pt layup,make,0,0,0,0,0
539261,The Arena Bay Lake Florida,October 11 2020,LAL,LAL,MIA,D. Howard - howardw01,3-pt jump shot,make,A. Caruso - carusal01,0,0,0,0
539262,The Arena Bay Lake Florida,October 11 2020,LAL,LAL,MIA,S. Hill - hillso01,3-pt jump shot,make,0,0,0,0,0
539263,The Arena Bay Lake Florida,October 11 2020,LAL,LAL,MIA,0,0,0,0,0,0,0,0


Transforming *ShotType* column to numeric

In [50]:
player_stats_2019_2020['ShotType'].value_counts()

ShotType
0                 336868
3-pt jump shot     78344
2-pt layup         55362
2-pt jump shot     51077
2-pt dunk          11740
2-pt hook shot      5874
Name: count, dtype: int64

In [51]:
player_stats_2019_2020['ShotType'] = player_stats_2019_2020['ShotType'].str.replace(r'-pt jump shot|-pt layup|-pt dunk|-pt hook shot','', regex=True).astype(float)
player_stats_2019_2020 = player_stats_2019_2020[(player_stats_2019_2020['ShotOutcome'] != 'miss') & (player_stats_2019_2020['FreeThrowOutcome'] != 'miss')]
player_stats_2019_2020.head()

,Location,Date,WinningTeam,AwayTeam,HomeTeam,Shooter,ShotType,ShotOutcome,Assister,Rebounder,FreeThrowShooter,FreeThrowOutcome,TurnoverPlayer
0,Scotiabank Arena Toronto Canada,October 22 2019,TOR,NOP,TOR,0,NaN,0,0,0,0,0,0
2,Scotiabank Arena Toronto Canada,October 22 2019,TOR,NOP,TOR,0,NaN,0,0,D. Favors - favorde01,0,0,0
3,Scotiabank Arena Toronto Canada,October 22 2019,TOR,NOP,TOR,D. Favors - favorde01,2.0,make,0,0,0,0,0
5,Scotiabank Arena Toronto Canada,October 22 2019,TOR,NOP,TOR,0,NaN,0,0,J. Redick - redicjj01,0,0,0
7,Scotiabank Arena Toronto Canada,October 22 2019,TOR,NOP,TOR,0,NaN,0,0,F. VanVleet - vanvlfr01,0,0,0


Filling missing values in *ShotType* column with 0

In [52]:
player_stats_2019_2020['ShotType'].fillna(0, inplace = True)
player_stats_2019_2020['ShotType']

C:\Users\IFE\AppData\Local\Temp\ipykernel_2312\1666426506.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_stats_2019_2020['ShotType'].fillna(0, inplace = True)


0         0.0
2         0.0
3         2.0
5         0.0
7         0.0
         ... 
539260    2.0
539261    3.0
539262    3.0
539263    0.0
539264    0.0
Name: ShotType, Length: 417979, dtype: float64

Aggregating the player stats to create the dataframe we will work with

In [59]:
player_stats_2019_2020['Date'] = pd.to_datetime(player_stats_2019_2020['Date'])
NBA2k19_pts_df = player_stats_2019_2020.groupby(['Shooter', 'Date'])['ShotType'].sum().reset_index()
NBA2k19_pts_df = NBA2k19_pts_df[NBA2k19_pts_df['Shooter'] != 0]
NBA2k19_pts_df

C:\Users\IFE\AppData\Local\Temp\ipykernel_2312\474887659.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_stats_2019_2020['Date'] = pd.to_datetime(player_stats_2019_2020['Date'])


,Shooter,Date,ShotType
193,A. Aminu - aminual01,2019-10-23,3.0
194,A. Aminu - aminual01,2019-10-28,7.0
195,A. Aminu - aminual01,2019-10-30,8.0
196,A. Aminu - aminual01,2019-11-01,3.0
197,A. Aminu - aminual01,2019-11-05,8.0
...,...,...,...
21225,Z. Williamson - willizi01,2020-07-30,12.0
21226,Z. Williamson - willizi01,2020-08-01,6.0
21227,Z. Williamson - willizi01,2020-08-03,18.0
21228,Z. Williamson - willizi01,2020-08-06,20.0


In [60]:
melted_df = pd.melt(player_stats_2019_2020[['Assister', 'Rebounder', 'TurnoverPlayer', 'FreeThrowShooter', 'Date']], id_vars='Date', value_vars=['Assister', 'Rebounder', 'TurnoverPlayer', 'FreeThrowShooter'], var_name='Role', value_name='Player')

# Group by 'date', 'player', and 'role', then calculate the count of occurrences for each group
NBA2k19_rebounds_df = melted_df.groupby(['Date', 'Player', 'Role']).size().unstack(fill_value=0).reset_index()
NBA2k19_rebounds_df = NBA2k19_rebounds_df[NBA2k19_rebounds_df['Player'] != 0]
NBA2k19_rebounds_df

Role,Date,Player,Assister,FreeThrowShooter,Rebounder,TurnoverPlayer
1,2019-10-22,A. Bradley - bradlav01,0,0,3,2
2,2019-10-22,A. Davis - davisan02,5,9,9,3
3,2019-10-22,B. Ingram - ingrabr01,5,4,5,2
4,2019-10-22,D. Favors - favorde01,2,0,7,1
5,2019-10-22,D. Green - greenda02,0,1,7,0
...,...,...,...,...,...,...
23487,2020-10-11,M. Morris - morrima02,0,0,2,0
23488,2020-10-11,R. Rondo - rondora01,4,0,4,4
23489,2020-10-11,S. Hill - hillso01,1,0,0,0
23490,2020-10-11,T. Herro - herroty01,4,0,3,4


In [75]:
NBA2k19 = pd.merge(NBA2k19_pts_df, NBA2k19_rebounds_df, left_on = ['Shooter', 'Date'], right_on = ['Player', 'Date'], how = 'outer')
NBA2k19['player_name'] = NBA2k19['Shooter'].combine_first(NBA2k19['Player'])
NBA2k19.drop(['Shooter', 'Player'], axis = 1, inplace = True)
NBA2k19 = NBA2k19[NBA2k19['player_name'] != 'Team']
NBA2k19.rename(columns = {'ShotType':'FG Points', 'Assister':'Assists', 'FreeThrowShooter':'FT Points', 'Rebounder':'Rebounds', 'TurnoverPlayer':'Turnovers',
                'player_name':'Player'}, inplace = True)
NBA2k19.fillna(0, inplace = True)
NBA2k19['Total Points'] = NBA2k19['FG Points'] + NBA2k19['FT Points']
NBA2k19 = NBA2k19[['Date', 'Player', 'Total Points', 'FG Points', 'FT Points', 'Rebounds', 'Assists', 'Turnovers']] # reorder columns
NBA2k19

,Date,Player,Total Points,FG Points,FT Points,Rebounds,Assists,Turnovers
0,2019-10-23,A. Aminu - aminual01,5.0,3.0,2.0,7.0,0.0,1.0
1,2019-10-28,A. Aminu - aminual01,7.0,7.0,0.0,5.0,3.0,1.0
2,2019-10-30,A. Aminu - aminual01,9.0,8.0,1.0,8.0,1.0,1.0
3,2019-11-01,A. Aminu - aminual01,3.0,3.0,0.0,5.0,1.0,2.0
4,2019-11-05,A. Aminu - aminual01,13.0,8.0,5.0,5.0,1.0,2.0
...,...,...,...,...,...,...,...,...
23573,2020-10-06,D. Howard - howardw01,0.0,0.0,0.0,2.0,0.0,2.0
23575,2020-10-09,A. Iguodala - iguodan01,0.0,0.0,0.0,6.0,2.0,1.0
23576,2020-10-09,D. Howard - howardw01,2.0,0.0,2.0,2.0,0.0,1.0
23577,2020-10-09,M. Morris - morrima02,0.0,0.0,0.0,1.0,0.0,3.0


In [55]:
injuries = pd.read_csv('injuries_2010-2020.csv')
injuries.head()

,Date,Team,Acquired,Relinquished,Notes
0,2010-10-03,Bulls,NaN,Carlos Boozer,fractured bone in right pinky finger (out inde...
1,2010-10-06,Pistons,NaN,Jonas Jerebko,torn right Achilles tendon (out indefinitely)
2,2010-10-06,Pistons,NaN,Terrico White,broken fifth metatarsal in right foot (out ind...
3,2010-10-08,Blazers,NaN,Jeff Ayres,torn ACL in right knee (out indefinitely)
4,2010-10-08,Nets,NaN,Troy Murphy,strained lower back (out indefinitely)
